In [56]:
# Jugador que se puede mover en cualquier lugar 
from dataclasses import dataclass, field
from typing import Dict, Union, List
import random as rd
from abc import ABC, abstractmethod

@dataclass
class Tiles(ABC):
    Name: str
    Position: List[int]
    def __post_init__(self):
        self.Position = self.Position

    def __repr__(self):
        return f"'{self.Name}'"



    @abstractmethod
    def Move(self):
        pass

    def CheckMovement(self, FuturePosition: int):
        if FuturePosition < 0 or FuturePosition > Grid.Size - 1:
            print('Invalid Movement')
            return False
        return True
    

@dataclass
class Player(Tiles):
    Health: int = 100
    Inventory= {
        "Medicine": 0,
        "Weapons": 0
    }

    def __post_init__(self):
        self.Position = self.Position

    def GetInventory(self):
        return (f'{self.Name} has {self.Inventory["Medicine"]} Medicines and {self.Inventory["Weapons"]} Weapons')
    
    def Move(self, Direction: str):
        if Direction.lower() == 'up' and self.CheckMovement(self.Position[0] - 1):
            self.Position[0] -= 1
        elif Direction.lower() == 'down' and self.CheckMovement(self.Position[0] + 1):
            self.Position[0] += 1
        elif Direction.lower() == 'left' and self.CheckMovement(self.Position[1] -1):
            self.Position[1] -= 1
        elif Direction.lower() == 'right' and self.CheckMovement(self.Position[1] + 1):
            self.Position[1] += 1
        else:
            print('Invalid Direction')
            return
        print(f'{self.Name} moved to {self.Position}')

    def __repr__(self):
        return f"'{self.Name}'"

    

@dataclass
class InventoryObjects(Tiles):
    
    def  UpdateInventory(self, Player: Player):
        pass

@dataclass
class Medicine(InventoryObjects):
    HealRegen: int
    def Move(self):
        pass
    def UpdateInventory(self, Player: Player):
        if Player.Health < 100:
            Player.Health += 10
            print(f'{Player.Name} has been healed by 10 points')
        else:
            Player.Inventory['Medicine'] += 1
            print(f'{Player.Name} has picked up a Medicine')
    def __repr__(self):
        return f"'{self.Name}'"

@dataclass
class Weapons(InventoryObjects):
    Damage: int = 5
    def Move(self):
        pass
    def UpdateInventory(self, Player: Player):
        Player.Inventory['Weapons'] += 1
        print(f'{Player.Name} has picked up a Weapon')

    def __repr__(self):
        return f"'{self.Name}'"

    
@dataclass
class Wall(Tiles):
    
    def Move(self):
        pass

    def __repr__(self):
        return f"'{self.Name}'"


@dataclass
class Enemies(Tiles):
    AttackPoints: int

    def Move(self):
        RandomDirection = rd.choice(['up', 'down', 'left', 'right'])
        if RandomDirection == 'up' and self.CheckMovement(self.Position[0] - 1):
            self.Position[0] -= 1
        elif RandomDirection == 'down' and self.CheckMovement(self.Position[0] + 1):
            self.Position[0] += 1
        elif RandomDirection == 'left' and self.CheckMovement(self.Position[1] -1):
            self.Position[1] -= 1
        elif RandomDirection == 'right' and self.CheckMovement(self.Position[1] + 1):
            self.Position[1] += 1
        else:
            print('Debug: Invalid Direction for enemy')
            return
        
    def __repr__(self):
        return f"'{self.Name}'"   
    






@dataclass
class Grid:
    Size: int = 15
    Positions: List = field(default_factory=list)
    Walls: List = field(default_factory=list)



    def __post_init__(self):
        self.grid = [[' ' for _ in range(self.Size)] for _ in range(self.Size)] 

    def AddPlayer(self, Player: Player):
        self.Positions.append(Player)

    
    def MoveEnemies(self):
        for Enemy in self.Positions:
            if isinstance(Enemy, Enemies):
                Enemy.Move()

    
    def Environment(self):
        for _ in range(10):    
            WallPosition = [rd.randint(0, self.Size - 1), rd.randint(0, self.Size - 1)]
            self.Walls.append(Wall('W', WallPosition))
            self.Positions.append(Medicine('M', [rd.randint(0, self.Size - 1), rd.randint(0, self.Size - 1)], 7))
            self.Positions.append(Weapons('A', [rd.randint(0, self.Size - 1), rd.randint(0, self.Size - 1)], 10))
            self.Positions.append(Enemies('E', [rd.randint(0, self.Size - 1), rd.randint(0, self.Size - 1)], 10 ))




    def CreateEnvironment(self):
        self.Environment()
    



    def UpdatePosition(self):
            self.grid = [[' ' for _ in range(self.Size)] for _ in range(self.Size)] 
            for Position in self.Positions:
                    self.grid[Position.Position[0]][Position.Position[1]] = Position
            for Wall in self.Walls:
                self.grid[Wall.Position[0]][Wall.Position[1]] = Wall  


    def GetElement(self, Position: List[int]):
        return self.grid[Position[0]][Position[1]]

    def UpdateGrid(self):
        self.MoveEnemies()
        self.UpdatePosition()
        self.ShowGrid()

    def ShowGrid(self):
        for row in self.grid:
            print(row)

    
@dataclass
class Game:
    Grid = Grid()
    Elements = ["P", "W", "M"]



    def MapLimits(self, PlayerPos):
        if PlayerPos[0] < 0 or PlayerPos[0] > 14 or PlayerPos[1] < 0 or PlayerPos[1] > 14:
            print('Movement blocked by map limits')
            return False
        return True
        

    def GetElements(self, Player: Player, Direction: str):
        PosibbleNewPos = {
            "up" : [-1,0],
            "down" : [1,0],
            "left" : [0,-1],
            "right" : [0,1],
        }
        pos = PosibbleNewPos[Direction]
        new_position = [Player.Position[0] + pos[0], Player.Position[1] + pos[1]]
        Element = self.Grid.GetElement(new_position)
        
        if isinstance(InventoryObjects):
            Element.UpdateInventory(Player)
            Player.Move(Direction)
            self.Grid.Positions.remove(Element)


    def MovePlayer(self, Player: Player, Direction: str):
        PosibbleNewPos = {
            "up" : [-1,0],
            "down" : [1,0],
            "left" : [0,-1],
            "right" : [0,1],
        }
        pos = PosibbleNewPos[Direction]
        new_position = [Player.Position[0] + pos[0], Player.Position[1] + pos[1]]

        if  self.MapLimits(new_position):
        
            if  isinstance(self.Grid.GetElement(new_position) ,Wall):
                print('Movement blocked by a wall')
            elif isinstance(self.Grid.GetElement(new_position) ,Medicine):
                self.GetElements(Player, Direction)
            elif isinstance(self.Grid.GetElement(new_position) , Weapons):
                self.GetElements(Player, Direction)

            else: 
                Player.Move(Direction)
                print(self.Grid.GetElement(new_position))
        else:
            print('Invalid Movement')



        



In [57]:
Juego = Game()
Player1 = Player('P', [2,2])
Juego.Grid.AddPlayer(Player1)
Juego.Grid.CreateEnvironment()


In [60]:
Juego.MovePlayer(Player1, 'right')
print(Player1.GetInventory())
Juego.Grid.UpdateGrid()

P moved to [3, 4]
 
P has 1 Medicines and 0 Weapons
[' ', ' ', 'M', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'A', 'E', ' ', ' ']
[' ', ' ', 'A', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'M', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', 'P', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'E', ' ', ' ', ' ', ' ', ' ', 'E', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', 'W', ' ', ' ', ' ', ' ', ' ', ' ', 'E', ' ', ' ', 'A']
[' ', ' ', 'W', 'W', ' ', ' ', ' ', ' ', 'E', ' ', 'W', 'A', ' ', 'E', ' ']
[' ', ' ', ' ', 'A', ' ', ' ', ' ', 'E', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['M', ' ', ' ', ' ', ' ', ' ', 'A', 'M', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', 'W', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'A', ' ', ' ', ' ', ' ', ' ']
[' ', 'W', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'W', ' ']
[' ', 'M', ' ', 'M', ' ', ' ', 'W', ' ', ' ', ' ', 'E', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', 

In [43]:
new_position = [Player1.Position[0] + 0, Player1.Position[1] - 1]

ola = Juego.GetElements(Player1, 'left')

print(ola)


None
